In this notebook, we generate Erdős–Rényi graphs, draw random subsets and check if they resolve the graph. We do this for every subset size from 1 to $n$ (the size of the graph) and estimate, experimentally, the probability that a subset of a certain size solve the graph.

In [1]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import sys
sys.path.append('../')
from helpers import *
import plotly.graph_objs as go

In [84]:
n = 50 # Size of the graph
p = 0.9 # Edge probability
nb_graph = 10 # nb of different graphs used for the simulations
s = {}
nb_of_iters = 1000 # nb of simulations for each subset size

# can be used to speed up simulations as we know the theoritical value
lower_bound = max([int(tvalue_case_1_2_p_const(n,p)) - 50, 0])
upper_bound = min([n, int(tvalue_case_1_2_p_const(n,p) + 50)])

In [85]:
solutions = {}
random.seed(0)
for _ in range(nb_graph):
    # Generate the random graph and compute shortest paths
    G = nx.erdos_renyi_graph(n, p)
    length = dict(nx.all_pairs_shortest_path_length(G))
    
    for nb in range(0, lower_bound):
        solutions[nb] = solutions.get(nb, 0) + 0  
    for nb in tqdm(range(lower_bound, upper_bound)): # G.number_of_nodes()
        num_nodes = nb # Number of nodes to sample
        node_list = list(G.nodes())
        count = 0
        for i in range(nb_of_iters):
            nodes = set(random.sample(node_list, num_nodes)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        solutions[nb] =  solutions.get(nb, 0) + (count / nb_of_iters)
    for nb in range(upper_bound, n):
            solutions[nb] = solutions.get(nb, 0) + 1

for i in range(n):
    solutions[i] = solutions[i] / nb_graph

100%|███████████████████████████████████████████| 50/50 [00:08<00:00,  5.71it/s]


In [75]:
#with open('simulations/sharp_n{}_p{}_i{}_g{}.pickle'.format(n, p, nb_of_iters, nb_graph), 'wb') as file:
    #pickle.dump(solutions, file, protocol=pickle.HIGHEST_PROTOCOL)

In [76]:
with open("sharp_n1000_p0.5_i800.pickle", "rb") as file:
    solutions = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'sharp_n1000_p0.5_i800.pickle'

In [86]:
# Define your data
x = list(solutions.keys())
y = list(solutions.values())

# Define the trace for the scatter plot
trace = go.Scatter(x=x, y=y, mode='markers+lines', name='Simulations')

# Define the trace for the vertical line
vertical_line = go.Scatter(x=[tvalue_case_1_2_p_const(n, p), tvalue_case_1_2_p_const(n, p)], y=[0, 1], mode='lines', line=dict(color='red'), name='Theoretical value')

# Define the layout
layout = go.Layout(#title='Probability of resolving the graph as a function of the subset cardinality', 
                   #title_x=0.5,
                   font=dict(size=16),
                   xaxis=dict(title='Cardinality of the subset'), 
                   yaxis=dict(title='Probability of resolving the graph'),
                   legend=dict(x=0.75, y=0.05, orientation='v', borderwidth=2))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace, vertical_line], layout=layout)
#fig.update_layout(legend_itemwidth=100)
# Show the figure
fig.show()